In [ ]:
#| default_exp diet_plots

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Tuple

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter

In [ ]:
#| export
from pheno_utils.timeseries_plots import filter_df, format_xticks
from pheno_utils.config import DEFAULT_CMAP


def plot_meals_bars(
    diet_log: pd.DataFrame, 
    x: str='collection_timestamp',
    participant_id: int=None, 
    array_index: int = None,
    time_range: Tuple[str, str]=None, 
    meals: bool=True,
    summary: bool=False,
    units: List[str]=['kcal', 'g', 'mg'],
    legend: bool=True,
    bar_width=np.timedelta64(15, 'm'),
    cmap: str=DEFAULT_CMAP,
    alpha: float=0.7,
    ax: plt.Axes=None,
    figsize: Tuple[float, float]=(14, 3),
):
    """
    Plot a stacked bar chart representing nutrient intake for each meal over time.

    Args:
        diet_log (pd.DataFrame): The dataframe containing the diet log data, with columns for timestamps, nutrients, and other measurements.
        participant_id (Optional[int]): The participant's ID to filter the diet log. If None, no filtering is done. Default is None.
        date_range (Optional[Tuple[str, str]]): A tuple of strings representing the start and end dates for filtering the data. Format should be 'YYYY-MM-DD'. Default is None.
        meals (bool): If True, includes individual meals in the plot. Default is True.
        summary (bool): If True, includes a daily summary in the plot. Default is False.
        units (List[str]): A list of strings representing the units to be included in the plot. Default is ['kcal', 'g', 'mg'].
        x_col (str): The name of the column in `diet_log` representing the x-axis variable, such as timestamps. Default is 'collection_timestamp'.
        bar_width (np.timedelta64): The width of the bars representing each meal on the time axis. Default is 15 minutes.
        ax (Optional[plt.Axes]): The Matplotlib axis on which to plot the bar chart. If None, a new axis is created. Default is None.

    Returns:
        None: The function creates a stacked bar chart on the specified or newly created axis.
    """
    # Prepare the data for plotting
    df, grouped_nutrients = prepare_meals(
        diet_log,
        participant_id=participant_id,
        array_index=array_index,
        time_range=time_range,
        return_meals=meals,
        return_summary=summary,
        return_units=units,
        x_col=x,
        )
    print(df)

    if ax is None:
        fig, ax = plt.subplots(len(grouped_nutrients), 1, figsize=(figsize[0], figsize[1] * len(grouped_nutrients)), sharex=True)
        if len(grouped_nutrients) == 1:
            ax = [ax]

    colors = sns.color_palette(cmap, len(grouped_nutrients))

    # Calculate the width in time units
    bar_width_in_days = bar_width / np.timedelta64(1, 'D')

    # Stacked bar plots for grouped nutrients
    for idx, (unit, nutrients) in enumerate(grouped_nutrients.items()):
        bottom = pd.Series([0] * len(df))
        for nut in nutrients:
            ax[idx].bar(
                df[x], df[nut], bottom=bottom, width=bar_width_in_days,
                color=colors[0], alpha=alpha, label=nut)
            bottom += df[nut]
        ax[idx].set_ylabel(f'Nutrients ({unit})')
        if legend:
            ax[idx].legend(title='Nutrients')
        ax[idx].grid(True)

    # Set x-tick labels for the bottom and top axes
    format_xticks(ax[-1], df[x])
    secax = ax[0].secondary_xaxis('top')
    secax.set_xticks(df[x])
    secax.set_xticklabels(df['food_items'], ha='center', fontsize=9)

    if ax is None:
        plt.tight_layout()
        plt.show()


def plot_meals_lollipop(
    diet_log: pd.DataFrame, 
    x: str='collection_timestamp',
    y: str='calories_kcal',
    size: str='weight_g', 
    participant_id: int=None, 
    array_index: int=None,
    time_range: Tuple[str, str]=None, 
    meals: bool=True,
    summary: bool=False,
    legend: bool=True,
    size_scale: float=1,
    cmap: str=DEFAULT_CMAP,
    alpha: float=0.7,
    ax: plt.Axes=None,
    figsize: Tuple[float, float] = (12, 6),
):
    """
    Plot a lollipop chart with pie charts representing nutrient composition for each meal.
    NOTE: If you intend to change `xlim` after plotting, you must also provide `date_range`.

    Args:
        diet_log (pd.DataFrame): The dataframe containing the diet log data, with columns for timestamps, nutrients, and other measurements.
        x (str): The name of the column in `diet_log` representing the x-axis variable, such as timestamps. Default is 'collection_timestamp'.
        y (str): The name of the column in `diet_log` representing the y-axis variable, such as calories. Default is 'calories_kcal'.
        size (str): The name of the column in `diet_log` representing the size of the pie charts. Default is 'weight_g'.
        participant_id (Optional[int]): The participant's ID to filter the diet log. If None, no filtering is done. Default is None.
        time_range (Optional[Tuple[str, str]]): A tuple of strings representing the start and end dates for filtering the data. Format should be 'YYYY-MM-DD'. Default is None.
        meals (bool): If True, includes individual meals in the plot. Default is True.
        summary (bool): If True, includes a daily summary in the plot. Default is False.
        legend (bool): If True, includes a legend in the plot. Default is True.
        size_scale (float): The scaling factor for the size of the pie charts. Default is 1.
        cmap (str): The color map to use for the pie slices. Default is 'tab20'.
        alpha (float): The transparency of the pie slices. Default is 0.7.
        ax (Optional[plt.Axes]): The Matplotlib axis on which to plot the lollipop chart. If None, a new axis is created. Default is None.
        figsize (Tuple[float, float]): The size of the figure to create. Default is (12, 6).

    Returns:
        None: The function creates a lollipop plot with pie charts on the specified or newly created axis.
    """
    # Prepare the data for plotting
    df, grouped_nutrients = prepare_meals(
        diet_log,
        participant_id=participant_id,
        array_index=array_index,
        time_range=time_range,
        return_meals=meals,
        return_summary=summary,
        return_units=['kcal', 'g', 'mg'],
        x_col=x,
        )

    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize)

    # Convert nutrients in mg to grams
    for nut in grouped_nutrients['mg']:
        df[nut.replace('_mg', '_g')] = df[nut] / 1000
        grouped_nutrients['g'] += [nut.replace('_mg', '_g')]

    pie_nuts = [nut for nut in grouped_nutrients['g'] if nut != 'weight_g']

    # Calculate unknown component and ensure all values are non-negative
    df['other_g'] = (df['weight_g'] - df[pie_nuts].sum(axis=1)).clip(lower=0)
    # pie_nuts += ['other_g']

    # Pre-set the x-axis limits based on the range of timestamps
    if time_range is None:
        min_x = mdates.date2num(df[x].min())
        max_x = mdates.date2num(df[x].max())
    else:
        min_x = mdates.date2num(pd.to_datetime(time_range[0]))
        max_x = mdates.date2num(pd.to_datetime(time_range[1]))

    # Pre-set the y-axis limits based on the range of the y-axis column
    min_y = 0  # df[y_col].min()
    max_y = df[y].max()

    # Calculate the aspect ratio between the x and y axes
    # This is necessary to avoid distortion of the (circular) pie charts
    x_range = max_x - min_x
    y_range = max_y - min_y
    aspect_ratio = x_range / y_range
    y_delta = 0.1 * y_range

    # Scale the y-axis to match the aspect ratio of the x-axis
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y * aspect_ratio, (max_y + y_delta) * aspect_ratio)

    # Custom formatter to adjust the y-ticks back to the original scale
    def ytick_formatter(y, pos):
        return f'{y / aspect_ratio:.0f}'

    # Plotting the lollipop plot with pies using absolute figure coordinates
    for idx, row in df.iterrows():
        # Pie chart parameters
        size_value = np.sqrt(row[size]) * aspect_ratio * size_scale
        position = mdates.date2num(row[x])
        y_value = row[y] * aspect_ratio  # Scale y-value

        # Plot the stem (lollipop stick)
        ax.plot([position, position], [0, y_value], color='gray', lw=1, zorder=1)

        # Plot the pie chart in figure coordinates (no distortion)
        wedges = draw_pie_chart(ax, position, y_value, row[pie_nuts].fillna(0.).values, size_value, cmap, alpha)
        # # Plot the pie chart using the calculated position and scaled radius
        # ax.pie(row[pie_nuts].fillna(0.).values, radius=size, center=(position, y_value), startangle=90,
        #        wedgeprops=dict(edgecolor='none'), normalize=True, colors=sns.color_palette(cmap, len(pie_nuts)),
        #        alpha=alpha)

    if legend:
        # Create a custom legend
        ax.legend(handles=wedges, labels= pie_nuts, loc='upper right', bbox_to_anchor=(1.2, 1), title="Nutrients")

    # Format x-axis to display dates properly
    ax.xaxis_date()
    ax.set_ylabel(y.replace('_', ' ').title())
    ax.grid(True)

    # Set y-ticks and x-ticks
    ax.yaxis.set_major_formatter(FuncFormatter(ytick_formatter))
    ylim = ax.get_ylim()
    yticks = np.arange(0, ylim[1] / aspect_ratio, 100, dtype=int)
    ax.set_yticks(yticks * aspect_ratio)
    ax.set_yticklabels(yticks)

    format_xticks(ax, df[x])

    secax = ax.secondary_xaxis('top')
    secax.set_xticks(df[x])
    secax.set_xticklabels(df['food_items'], ha='center', fontsize=9)

    if ax is None:
        plt.tight_layout()
        plt.show()


def prepare_meals(
    diet_log: pd.DataFrame,
    participant_id: int=None,
    array_index: int=None,
    time_range: Tuple[str, str]=None,
    return_meals: bool = True,
    return_summary: bool = False,
    return_units: List[str]=['kcal', 'g', 'mg'],
    x_col: str='collection_timestamp'
) -> pd.DataFrame:
    """
    Prepare the diet log data for plotting meals and/or daily summaries.

    Args:
        diet_log (pd.DataFrame): The dataframe containing the diet log data, with columns for timestamps, nutrients, and other measurements.
        participant_id (Optional[int]): The participant's ID to filter the diet log. If None, no filtering is done. Default is None.
        array_index (Optional[int]): The array index to filter the diet log. If None, no filtering is done. Default is None.
        time_range (Optional[Tuple[str, str]]): A tuple of strings representing the start and end dates for filtering the data. Format should be 'YYYY-MM-DD'. Default is None.
        return_meals (bool): If True, includes individual meals in the plot. Default is True.
        return_summary (bool): If True, includes a daily summary in the plot. Default is False.
        return_units (List[str]): A list of strings representing the units to be included in the plot. Default is ['kcal', 'g', 'mg'].
        x_col (str): The name of the column in `diet_log` representing the x-axis variable, such as timestamps. Default is 'collection_timestamp'.

    Returns:
        pd.DataFrame: A dataframe containing the prepared data for plotting.
    """
    diet_log = filter_df(diet_log, participant_id, array_index, time_range, x_start=x_col, x_end=x_col)\
        .set_index(diet_log.index.names)

    units = extract_units(diet_log.columns)
    grouped_nutrients = {}
    agg_dict = {}
    for nut, unit in units.items():
        if unit not in return_units:
            continue
        if unit not in grouped_nutrients:
            grouped_nutrients[unit] = []
        grouped_nutrients[unit].append(nut)
        agg_dict[nut] = 'sum'
    nut_list = list(agg_dict.keys())
    agg_dict['short_food_name'] = lambda x: '\n'.join(x)

    df = diet_log\
        .dropna(subset=['short_food_name'])\
        .drop_duplicates()\
        .groupby([x_col])\
        .agg(agg_dict)\
        .rename(columns={'short_food_name': 'food_items'})\
        .reset_index()

    if return_summary:
        # Add daily summary by grouping by date and summing up the nutrients
        daily_df = df.groupby(df[x_col].dt.date)[nut_list]\
            .sum().reset_index()
        daily_df['food_items'] = daily_df[x_col].astype('string') + '\nDaily Summary'
        daily_df[x_col] = pd.to_datetime(daily_df[x_col] + pd.Timedelta(hours=24))
        if time_range is not None:
            daily_df = daily_df[(time_range[0] <= daily_df[x_col]) & (daily_df[x_col] <= time_range[1])]
        if return_meals:
            # God knows why, but the two refuse to concat without this
            df = pd.DataFrame(np.vstack([df, daily_df]), columns=df.columns)
        else:
            df = daily_df

    return df, grouped_nutrients


def extract_units(column_names: List[str]) -> dict:
    units = {}
    for col in column_names:
        if '_' in col:
            unit = col.split('_')[-1]
            units[col] = unit
        else:
            units[col] = 'unknown'
    return units


def draw_pie_chart(
    ax: plt.Axes, 
    x: float, 
    y: float, 
    data: List[float], 
    size: float, 
    cmap: str = DEFAULT_CMAP,
    alpha: float = 0.7,
):
    """
    Draw a pie chart as an inset (in absolute figure coordinates) within the given axes
    at the specified data coordinates.
    What this solves is the issue of y-axis and x-axis scaling being different, which
    distorts the pie chart when drawn directly on the axes.

    Args:
        ax (plt.Axes): The axis on which to draw the pie chart.
        x (float): The x-coordinate in data coordinates where the pie chart's center will be placed.
        y (float): The y-coordinate in data coordinates where the pie chart's center will be placed.
        data (List[float]): The data values to be represented in the pie chart.
        size (float): The size (radius) of the pie chart in axes-relative coordinates.
        cmap (str): The color map to use for the pie slices.

    Returns:
        List[plt.Patch]: A list of wedge objects representing the pie chart slices.
    """
    # Convert the position from data coordinates to axes coordinates
    axes_coords = ax.transData.transform((x, y))
    axes_coords = ax.transAxes.inverted().transform(axes_coords)

    # Create a new inset axis to draw the pie, using axes-relative coordinates
    inset_ax = ax.inset_axes([axes_coords[0] - size, axes_coords[1] - size, 2 * size, 2 * size])

    # Plot the pie chart using the calculated position and scaled radius
    colors = [(r, g, b, alpha) for r, g, b in sns.color_palette(cmap, len(data))]
    wedges, _ = inset_ax.pie(data, radius=1, startangle=90, wedgeprops=dict(edgecolor='none'), normalize=True,
                             colors=colors)

    # Hide the axes for the inset (pie chart)
    inset_ax.set_axis_off()

    return wedges


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()